In [33]:
from langchain.prompts import PromptTemplate
from langchain.retrievers import AzureCognitiveSearchRetriever
from langchain.chat_models import AzureChatOpenAI
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.chains import LLMChain
from langchain.tools import BaseTool
from langchain.agents import ZeroShotAgent, AgentExecutor
from functools import reduce
import pandas as pd
import os

os.environ["AZURE_COGNITIVE_SEARCH_SERVICE_NAME"] = "search-service01"
os.environ["AZURE_COGNITIVE_SEARCH_INDEX_NAME"] = "public-data-index-alt"
os.environ["AZURE_COGNITIVE_SEARCH_API_KEY"] = "73Swa5YqUR5IRMwUIqOH6ww2YBm3SveLv7rDmZVXtIAzSeBjEQe9"

retriever_public = AzureCognitiveSearchRetriever(content_key='content')
print(retriever_public)

os.environ["AZURE_COGNITIVE_SEARCH_INDEX_NAME"] = "private-index-test"
retriever_private = AzureCognitiveSearchRetriever(content_key='content')
print(retriever_private)

service_name='search-service01' index_name='public-data-index-alt' api_key='73Swa5YqUR5IRMwUIqOH6ww2YBm3SveLv7rDmZVXtIAzSeBjEQe9' api_version='2020-06-30' aiosession=None content_key='content'
service_name='search-service01' index_name='private-index-test' api_key='73Swa5YqUR5IRMwUIqOH6ww2YBm3SveLv7rDmZVXtIAzSeBjEQe9' api_version='2020-06-30' aiosession=None content_key='content'


In [34]:
# Load up your LLM
llm = AzureChatOpenAI(
    openai_api_type="azure",
    openai_api_base='https://openai-nois-intern.openai.azure.com/',
    openai_api_version="2023-03-15-preview",
    deployment_name='test-1',
    openai_api_key=os.getenv('OPENAI_API_KEY'),
    temperature=0.5,
    max_tokens=300
)

history = []
history_entry = 1

def get_history_as_txt():
    txt = ""
    for i in history:
        txt += f"\n<|im_start|>user\n{i['user']}\n<|im_end|>\n"
        txt += f"<|im_start|>assistant\n{i['AI']}\n<|im_end|>"
        
    return txt

def add_to_history(user_msg, ai_msg):
    history.append({'user': user_msg, 'AI': ai_msg})

In [35]:
def save_history_to_file():
    global history_entry
    file = open(f"chatbot-cases/chat-{history_entry}.txt", 'w', encoding='utf-8')
    file.write(get_history_as_txt())
    
    file.close()
    
    history_entry += 1

In [36]:
print(llm)

verbose=False callbacks=None callback_manager=None tags=None client=<class 'openai.api_resources.chat_completion.ChatCompletion'> model_name='gpt-3.5-turbo' temperature=0.5 model_kwargs={} openai_api_key='400568d9a16740b88aff437480544a39' openai_api_base='https://openai-nois-intern.openai.azure.com/' openai_organization='' openai_proxy='' request_timeout=None max_retries=6 streaming=False n=1 max_tokens=300 deployment_name='test-1' openai_api_type='azure' openai_api_version='2023-03-15-preview'


In [44]:
keyword_templ = """<|im_start|>system
Given a sentence and a conversation about the companies New Ocean and NOIS, assistant will extract keywords based on the conversation and the sentence and translate them to Vietnamese if they're in English and vice versa.
Your output will be on one line, in both languages if possible and separated by commas. Do not duplicate keywords.
Assume the context is about the companies unless specified otherwise. Only return the keywords, do not output anything else.

EXAMPLE
Input: Ai chưa đóng tiền nước?
Output: tiền nước, water bill
Input: Was Pepsico a customer of New Ocean?
Output: Pepsico
Input: What is FASF?
Output: FASF
<|im_end|>
{context}
<|im_start|>user
Input: {question}
<|im_end|>
<|im_start|>assistant
Output:"""

query_prompt_template = """Below is a history of the conversation so far, and a new question asked by the user that needs to be answered by querying relevant company documents.
Generate a search query based on the conversation and the new question. 

Chat History:{chat_history}

Question:
{question}

Search query:
"""

template = """<|im_start|>system
Assistant is a friendly chatbot that helps the company employees and users with their questions about the companies New Ocean and NOIS in their language. Be brief in your answers.
Answer ONLY with the facts listed in the list of sources below. If there isn't enough information below, say you don't know. Do not generate answers that don't use the sources below. If asking a clarifying question to the user would help, ask the question.
If the user asks any questions not related to New Ocean or NOIS, apologize and request another question from the user. If the user greets you, respond accordingly.
If given a question with both languages present, ask the user which language they prefer.
Sources:
{summaries}
<|im_end|>
{context}
<|im_start|>user
{question}
<|im_end|>
<|im_start|>assistant
"""

In [45]:
qaChain = load_qa_with_sources_chain(llm=llm, chain_type="stuff", prompt=PromptTemplate.from_template(template))
keywordChain = LLMChain(llm=llm, prompt=PromptTemplate.from_template(keyword_templ))

In [46]:
def get_docs_using_keyword_string(keywords, ret):
    return ret.get_relevant_documents(keywords)[:5]

In [57]:
query = "NOIS có những dịch vụ gì?"
keywords = keywordChain({'context': get_history_as_txt(), 'question': query})

doc = get_docs_using_keyword_string(keywords['text'], retriever_public)

num = 1
for x in doc:
    x.metadata['source'] = f"doc-{num}"

    num += 1

result = qaChain({'input_documents': doc, 'question': query, 'context': get_history_as_txt()}, return_only_outputs=False)
    
add_to_history(query, result['output_text'])

In [58]:
print(keywords['text'])

NOIS, dịch vụ, services


In [59]:
result['output_text']

'NOIS cung cấp nhiều dịch vụ trong lĩnh vực sản xuất và dịch vụ gia công phần mềm CNTT, bao gồm các giải pháp như Supply Chain, Traceability, Warehouse Management và Factory Smart Forms & Checklists. Ngoài ra, NOIS còn cung cấp các giải pháp chìa khóa trao tay trong ngành Logistics như kích hoạt các dịch vụ tùy chỉnh và giúp doanh nghiệp tiết kiệm chi phí trong phát triển, thiết lập và bảo trì cơ sở hạ tầng CNTT riêng.'

In [60]:
print(doc)

[Document(page_content='t số trong sản xuất và dịch vụ gia công phần mềm CNTT. Nổi bật có thể kể đến các giải pháp như Supply Chain, Traceability, Warehouse Management…Một dự án khác của NOIS đang gây tiếng vang, thu hút sự quan tâm của rất nhiều doanh nghiệp đó là Factory Smart Forms & Checklists. 2 năm bùng nổ dịch bệnh Covid-19, nền\xa0 kinh tế thế giới và Việt Nam chịu nhiều ảnh hưởng sâu sắc. Sau đợt dịch thứ 4 bùng nổ tại Việt Nam, cuối năm 2021 là th�\n', metadata={'@search.score': 11.384831, 'metadata_storage_path': 'aHR0cHM6Ly9hY3NjaGF0Ym90bm9pc2ludGVybi5ibG9iLmNvcmUud2luZG93cy5uZXQvcHVibGljLWRhdGEtYWx0LzUyMC0xMS50eHQ1', 'source': 'doc-1'}), Document(page_content='ó khăn về nhân lực, nguồn vốn và duy trì. Hãy đến ngay với New Ocean Information System NOIS để được trải nghiệm dịch vụ tuyệt vời nhất.Thành lập từ năm 2010, trong suốt chặng đường phát triển NOIS không ngừng mở rộng nguồn tài nguyên công nghệ của mình, tạo ra những sản phẩm, giải pháp có tính ứng dụng cao, phù hợp 

In [56]:
history = []

In [28]:
get_history_as_txt()

"\n<|im_start|>user\nWhat is New Ocean?\n<|im_end|>\n<|im_start|>assistant\nNew Ocean is a company that provides solutions for quality control in the pharmaceutical and chemical industries, as well as automation systems. They recently participated in the VIMF 2023 exhibition, with their booth located at number 76 in the Innovation area. For more information, you can contact them through their hotline at 1900 0224 or email at customercare@new-ocean.com.vn.\n<|im_end|>\n<|im_start|>user\nWhat are New Ocean's products?\n<|im_end|>\n<|im_start|>assistant\nNew Ocean provides solutions for quality control in the pharmaceutical and chemical industries, as well as automation systems. However, I do not have specific information about their products beyond this. Would you like me to look up more information?\n<|im_end|>\n<|im_start|>user\nNo, it's fine.\n<|im_end|>\n<|im_start|>assistant\nAlright, let me know if you have any other questions related to New Ocean or their products.\n<|im_end|>\n<|

In [29]:
save_history_to_file()

In [24]:
doc_txt = ""
for x in doc:
    doc_txt += x.page_content
    
context = get_history_as_txt()
    
msg = f"""<|im_start|>system
Assistant helps the company employees and users with their questions about the companies New Ocean and NOIS. Be brief in your answers.
Answer ONLY with the facts listed in the list of sources below. If there isn't enough information below, say you don't know. Do not generate answers that don't use the sources below. If asking a clarifying question to the user would help, ask the question.
If the user asks any questions not related to New Ocean or NOIS, apologize and request another question from the user. If the user greets you, respond accordingly.
Sources:
{doc_txt}
<|im_end|>
{context}
"""

print(msg)
print(llm.get_num_tokens(msg))
print(f"Length of chat history: {llm.get_num_tokens(get_history_as_txt())}")

<|im_start|>system
Assistant helps the company employees and users with their questions about the companies New Ocean and NOIS. Be brief in your answers.
Answer ONLY with the facts listed in the list of sources below. If there isn't enough information below, say you don't know. Do not generate answers that don't use the sources below. If asking a clarifying question to the user would help, ask the question.
If the user asks any questions not related to New Ocean or NOIS, apologize and request another question from the user. If the user greets you, respond accordingly.
Sources:
Data Analytics, Data Engineering  & Machine Learning Portfolio
HomeData Analytics, Data Engineering  & Machine Learning Portfolio
// our portfolio
Data Analytics, Data Engineering  & Machine Learning

Data Analytics, Data Engineering  & Machine Learning Portfolio
HomeData Analytics, Data Engineering  & Machine Learning Portfolio
// our portfolio
Data Analytics, Data Engineering  & Machine Learning

APPLICATION OF

In [19]:
class PrivateDataTool(BaseTool):
    name = "Excel sheet getter tool"
    description = """use this tool when you need to get an information sheet from an excel file exclusive only to New Ocean and NOIS's employees.
    Query by inputting the sheet name. The excel file contains 4 sheets (case-sensitive) which are:
    - Training Plan: contains past and upcoming company training plans.
    - Company Trip_Team Building 2023: contains upcoming company trips.
    - Organization Chart: contains details regarding development teams, members, their positions and their managers.
    - May Drink Fee: contains info on people who have or haven't paid drink fee in May."""

    def _run(self, query: str):
        try:
            df = pd.read_excel("Data Private Chatbot.xlsx", sheet_name=query, skiprows=1)
            return df
        
        except Exception:
            return f"\nSheet {query} not found, the file contains only 4 sheets which are Training Plan, Organization Chart, Company Trip_Team Building 2023 and May Drink Fee. Try inputting the correct sheet name."
    
    def _arun(self, query: str):
        raise NotImplementedError("This tool does not support async")
        
class DoNothing(BaseTool):
    name = "Generic tool"
    description = """use this tool if you encounter any questions that are not related to New Ocean or NOIS."""
    
    def _run(self, query: str):
        return "No documents found.<|im_end|>"
    
    def _arun(self, query: str):
        raise NotImplementedError("This tool does not support async")

In [20]:
# Define query tool for agent
class DocumentQueryTool(BaseTool):
    name = "Document query tool"
    description = "use this tool when you need to get documents for questions related to New Ocean and NOIS."
    
    def _run(self, query: str):
        doc = retriever_public.get_relevant_documents(query)[:4]
        try:
            content = reduce(lambda x, y: x + y.page_content, doc, doc[0].page_content)
            return content
        except Exception:
            return "No documents found."
    
    def _arun(self, query: str):
        raise NotImplementedError("This tool does not support async")

        
prefix = """You are an intelligent assistant helping New Ocean and NOIS employees with company related questions. You will be provided with keywords that you can use for querying documents.
If you cannot answer using the queried documents, try using other provided tools. If you cannot get any results from tools, say you don't know. Do not use any other sources other than the ones you obtain from tools.
Mention which tool you used to get the result. Answer in the question's language. You have access to the following tools:"""
suffix = """Begin!

Keywords: {keywords}
Question: {input}
{agent_scratchpad}"""

tools = [DocumentQueryTool(), PrivateDataTool(), DoNothing()]

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["keywords", "input", "agent_scratchpad"],
)

llm_chain = LLMChain(llm=llm, prompt=prompt)

agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True, stop=["\nObservation:", "<|im_end|>"])
agent_chain = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors="Check your output and make sure it conforms!"
)

In [21]:
query = "Who hasn't paid the drink fee?"
keywords = keywordChain({'question': query, 'context': get_history_as_txt()})['text']
print(keywords)

res = agent_chain.run(input=query, keywords=keywords)
res

drink fee, phí đồ uống, paid


> Entering new  chain...
Thought: This question is related to the May Drink Fee sheet in the excel file.
Action: Excel sheet getter tool
Action Input: May Drink Fee
Observation:     STT                       Email        FullName  May Total  Thực thu  \
0     1            hung.bui@nois.vn   BÙI TUẤN HƯNG      68000       NaN   
1     2           hiep.dang@nois.vn   ĐẶNG DUY HIỆP      33542   33542.0   
2     3            tai.dang@nois.vn    ĐẶNG HỮU TÀI      56100   56100.0   
3     4            sang.dao@nois.vn   ĐÀO MINH SÁNG      24246       NaN   
4     5              bao.do@nois.vn     ĐỖ QUỐC BẢO     106246       NaN   
..  ...                         ...             ...        ...       ...   
60   61           phuc.tran@nois.vn   TRẦN VĂN PHÚC      13750   13750.0   
61   62  tai.vo@newoceaninfosys.com      VÕ TẤN TÀI      43696   43000.0   
62   63             hien.vo@nois.vn   VÕ THANH HIỂN      68150       NaN   
63   64              nam.vu@noi

'The people who haven\'t paid the drink fee yet have NaN values in the "Thực thu" column and "NaN" in the "Tình trạng" column.'

In [1]:
thingy = PrivateDataTool()
# llm.get_num_tokens(thingy._run('Organization Chart'))
thingy._run('Organization Chart')

NameError: name 'PrivateDataTool' is not defined

In [ ]:
summary_template = """<|im_start|>system
Assistant helps user summarize documents in one sentence. Output format will be:
1. Summary of document 1
2. Summary of document 2
etc
<|im_end|>

<|im_start|>user
{docs}
<|im_end|>
<|im_start|>assistant
"""

summaryChain = LLMChain(llm=llm, prompt=PromptTemplate.from_template(summary_template))

In [2]:
docuQuery = "Nghỉ phép"
doc = retriever_private.get_relevant_documents(docuQuery)[:3]

string = ""
for i in doc:
    string += i.page_content
    
res = summaryChain(string)
res['text']

NameError: name 'retriever_private' is not defined

In [3]:
llm.get_num_tokens(string)

NameError: name 'llm' is not defined

In [49]:
doc

[Document(page_content='\nĐiều 7. Nghỉ phép năm \n\n1. Người lao động có đủ 12 tháng làm việc tại công ty thì được nghỉ 12 ngày phép năm hưởng \n\nnguyên lương. \n\n2. Đối với trường hợp làm không đủ năm (làm việc dưới 12 tháng): \n\nLấy số ngày nghỉ hàng năm cộng với số ngày được nghỉ tăng thêm theo thâm niên (nếu có), chia \n\ncho 12 tháng, nhân với số tháng làm việc thực tế trong năm để tính thành số ngày được nghỉ hàng \n\nnăm. \n\nKết quả phép tính lấy tròn số hàng đơn vị, nếu phần thập phân lớn hơn hoặc bằng 0,5 thì làm tròn \n\nlên 01 đơn vị. \n\n3. Cứ 05 năm làm việc tại công ty thì số ngày nghỉ hàng năm được tăng thêm 01 ngày. \n\n4. Lịch nghỉ phép hàng năm: \n\n- Mỗi tháng người lao động được nghỉ 01 ngày, nếu không nghỉ thì ngày phép đó sẽ cộng dồn vào \n\nnhững tháng sau. Nhân viên cũng có thể sử dụng một lần phép năm nếu không ảnh hưởng đến \n\ncông việc \n\n- Khi người lao động muốn nghỉ phép phải làm đơn xin phép và được sự đồng ý của người quản \n\nlý trực tiếp hoặc Giá